# Save the merged model and test it

In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft

Hugging face for read :

In [2]:
!huggingface-cli login --token $'your token'

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
base_model = "aymanboufarhi/gemma2B-chat-bot-fstt"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch



model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

2024-05-28 15:46:48.775009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 15:46:48.775107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 15:46:48.937090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

adapter_config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/314M [00:00<?, ?B/s]

In [8]:
modelmerged = model.merge_and_unload()

In [ ]:
model

In [ ]:
modelmerged

Hugging face for write :

In [9]:
!huggingface-cli login --token $'your token'

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
modelmerged.push_to_hub("aymanboufarhi/gemma2B-chat-bot-fstt", use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aymanboufarhi/gemma2B-chat-bot-fstt/commit/8ddcda2bad062760af7ba0204b8a58493b1295b7', commit_message='Upload GemmaForCausalLM', commit_description='', oid='8ddcda2bad062760af7ba0204b8a58493b1295b7', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [11]:
prompt = '''<|system|>FSTT c'est la Faculté des Sciences et Techniques de Tanger 
<|user|> le nombre de départements 
<|assistant|>'''
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=500, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

<|system|>FSTT c'est la Faculté des Sciences et Techniques de Tanger 
<|user|> le nombre de départements 
<|assistant|> 9 
<|assistant|> GÉNIE INFORMATIQUE, GÉNIE CHIMIQUE, SCIENCES DE LA TERRE, GÉNIE MÉCANIQUE, SCIENCES DE LA VIE, GÉNIE ELECTRIQUE, TEC, PHYSIQUE, MATHÉMATIQUES </s>
<|assistant|>  Département de GÉNIE ELECTRIQUE </s>
<|assistant|>  Département de GÉNIE MÉCANIQUE </s>
<|assistant|>  Département de GÉNIE CHIMIQUE </s>
<|assistant|>  Département de SCIENCES DE LA TERRE </s>
<|assistant|>  Département de SCIENCES DE LA VIE </s>
<|assistant|>  Département de TEC </s>
<|assistant|>  Département de PHYSIQUE </s>
<|assistant|>  Département de MATHÉMATIQUES </s>
